In [1]:
#примеры из статьи https://habr.com/ru/post/531940/
from spacy.lang.ru import Russian
nlp = Russian()
doc = nlp("Съешь ещё этих мягких французских булок, да выпей чаю. А лучше 2.")
token = doc[0]
print(token.text)

Съешь


In [2]:
span = doc[3:6]
print(span.text)

мягких французских булок


In [3]:
print("is_alpha:    ", [(token,token.is_alpha) for token in doc])
print("is_punct:    ", [(token,token.is_punct) for token in doc])
print("like_num:    ", [(token,token.like_num) for token in doc])

is_alpha:     [(Съешь, True), (ещё, True), (этих, True), (мягких, True), (французских, True), (булок, True), (,, False), (да, True), (выпей, True), (чаю, True), (., False), (А, True), (лучше, True), (2, False), (., False)]
is_punct:     [(Съешь, False), (ещё, False), (этих, False), (мягких, False), (французских, False), (булок, False), (,, True), (да, False), (выпей, False), (чаю, False), (., True), (А, False), (лучше, False), (2, False), (., True)]
like_num:     [(Съешь, False), (ещё, False), (этих, False), (мягких, False), (французских, False), (булок, False), (,, False), (да, False), (выпей, False), (чаю, False), (., False), (А, False), (лучше, False), (2, True), (., False)]


In [4]:
#последние слова до точек
for token in doc:
    if token.i+1 < len(doc):
        next_token = doc[token.i+1]
        if next_token.text == ".":
            print(token.text)

чаю
2


In [5]:
#python -m spacy download en_core_web_sm

import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("New Apple MacBook set launch tomorrow")

for token in doc:
    token_text = token.text
    token_pos = token.pos_
    token_dep = token.dep_
    token_head = token.head.text
    print(f"{token_text:<12}{token_pos:<10}" \
          f"{token_dep:<10}{token_head:<12}")

New         PROPN     compound  MacBook     
Apple       PROPN     compound  MacBook     
MacBook     PROPN     nmod      launch      
set         VERB      amod      launch      
launch      NOUN      ROOT      launch      
tomorrow    NOUN      npadvmod  launch      


In [6]:
from spacy import displacy
displacy.render(doc, style='dep', jupyter=True)

In [7]:
#расшифровка названий
print(spacy.explain("aux"))
print(spacy.explain("PROPN"))


auxiliary
proper noun


In [8]:
#начальная форма слова
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("I saw a movie yesterday")
print(' '.join([token.lemma_ for token in doc]))


-PRON- see a movie yesterday


In [9]:
#именованные сущности
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("Apple is looking at buying U.K. startup for 1$ billion")
for ent in doc.ents:
    print(ent.text, ent.label_)

Apple ORG
U.K. GPE
1$ billion MONEY


In [10]:
print(spacy.explain("GPE"))
print(spacy.explain("ORG"))

Countries, cities, states
Companies, agencies, institutions, etc.


In [11]:
from spacy import displacy
displacy.render(doc, style='ent', jupyter=True)

In [12]:
#пример поиска
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)
pattern = [
    {"IS_DIGIT": True}, 
    {"LOWER": {"REGEX": "(fifa|icc)"}},
    {"LOWER": "cricket", "OP": "?"},
    {"LOWER": "world"},
    {"LOWER": "cup"}
]
matcher.add("fifa_pattern", None, pattern)
doc = nlp("2018 ICC Cricket World Cup: Afghanistan won!")
matches = matcher(doc)
for match_id, start, end in matches:
    matched_span = doc[start:end]
    print(matched_span)

2018 ICC Cricket World Cup


In [13]:
nlp = spacy.load("en_core_web_sm") # лучше брать модели побольше, например, en_core_web_md
doc1 = nlp("I like burgers")
doc2 = nlp("I like pizza")
print(doc1.similarity(doc2))

0.7049178282720564


C:\Miniconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  after removing the cwd from sys.path.


In [14]:
#Модуль SpaCy поддерживает ряд встроенных компонентов (токенизатор, выделение именованных сущностей), 
#но также позволяет определять свои собственные компоненты. По сути, компоненты – это последовательно 
#вызывающиеся функции, которые принимают на вход документ, изменяют его и отдают обратно. Новые компоненты
#можно добавлять с помощью атрибута add_pipe:
import spacy

def length_component(doc):
    doc_length = len(doc)
    print(f"This document is {doc_length} tokens long.")
    return doc

nlp = spacy.load("en_core_web_sm")
nlp.add_pipe(length_component, first=True)
print(nlp.pipe_names)
doc = nlp("This is a sentence.")

['length_component', 'tagger', 'parser', 'ner']
This document is 5 tokens long.
